In [1]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import distance_helpers
import numpy as np
import json
import cv2

In [2]:
inputfiles = ['petfinder_pca_11063_100.pickle','petfinder_dog_cnn_simple.pickle']

In [3]:
photo_ids = []
breedList = dict()
photoId2Index = dict()
with open("small_dog_data.json") as data_file:    
    dog_data = json.load(data_file)
counter = 0
for k,v in dog_data.iteritems():
    nBreeds = float(len(v["breeds"]))
    if len(v["breeds"]) > 1: continue
    breed = v["breeds"][0]
    photos = v["photos"]
    for photo in photos:
        imagename = "cutPetFinder/"+k+"_"+photo["id"]+".jpg"
        img = cv2.imread(imagename)
        if img is None: continue
        photo_ids.append(k)
        photoId2Index[k] = counter
        if not breedList.has_key(breed): breedList[breed] = []
        breedList[breed].append(counter)
        counter += 1
        break

In [4]:
print len(photo_ids),len(breedList)

1541 111


In [5]:
breedDistances = { key : [] for key in breedList.keys()}
sumBreedDistances = [0 for i in range(len(inputfiles))]

for i,inputfilename in enumerate(inputfiles):
    with open(inputfilename, 'rb') as handle:
        b = pickle.load(handle)
        coefficients = np.array(b.values())
        similarities = distance_helpers.getDistances(coefficients)
        avgForAll = distance_helpers.getAvgDistance(similarities)
        print inputfilename,coefficients.shape,avgForAll
        for k,v in breedList.iteritems():
            #print k,len(v)
            if len(v) < 20:
                breedDistances[k].append(0)
                continue
            idx = np.array(v)
            slimSimilarities = similarities[idx[:, None], idx]
            breedAvg = distance_helpers.getAvgDistance(slimSimilarities)
            breedDistances[k].append(breedAvg/avgForAll)
            sumBreedDistances[i] += breedAvg/avgForAll

petfinder_pca_11063_100.pickle (1541, 100) 0.0930056695018
petfinder_dog_cnn_simple.pickle (1541, 120) 0.00898506303884


In [6]:
print sumBreedDistances

[13.536538662564611, 13.402936305429302]
